In [3]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout, Reshape
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D 
from keras.utils import to_categorical

import matplotlib.pyplot as plt
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

2022-04-18 12:24:35.847922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [5]:
os.listdir('/home/data/datasets/sicong_bioz_bp/bioz_pickles/')

['75-5_75H.pkl', '75-4_75E.pkl', '75-1_75A.pkl', '75-8_75F.pkl']

In [6]:
file = open('/home/data/datasets/sicong_bioz_bp/bioz_pickles/75-5_75H.pkl', 'rb')
all_75h = pickle.load(file)
file.close()

In [7]:
all_75h.keys()

dict_keys(['session06_05212021', 'session05_0518_2021', 'session07', 'session01_0406_2021', 'session03_0510_2021', 'session04_0513_2021', 'session02_0413_2021'])

In [8]:
test00 = all_75h['session06_05212021']['test00-20210521 13_20_26']['BioZ_ch1']
test01 = all_75h['session06_05212021']['test01-20210521 13_20_26']['BioZ_ch1']


trial10_test00 = all_75h['session06_05212021']['test00-20210521 13_46_18']['BioZ_ch1']
trial11_test01 = all_75h['session06_05212021']['test01-20210521 13_46_18']['BioZ_ch1']

trial20_test00 = all_75h['session06_05212021']['test00-20210521 13_55_53']['BioZ_ch1']
trial21_test01 = all_75h['session06_05212021']['test01-20210521 13_55_53']['BioZ_ch1']

trial30_test00 = all_75h['session06_05212021']['test00-20210521 13_29_26']['BioZ_ch1']
trial31_test01 = all_75h['session06_05212021']['test01-20210521 13_29_26']['BioZ_ch1']


frames = [trial10_test00, trial11_test01]

data = pd.concat(frames)
data = data.fillna(method="ffill")

In [9]:
X = data.iloc[:,:-4]
y = data.iloc[:, -4] - data.iloc[:, -3]

X_train = X[:int(0.8*len(X))]
X_test = X[int(0.8*len(X)):]

y_train = y[:int(0.8*len(y))]
y_test = y[int(0.8*len(y)):]

In [10]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = tf.reduce_mean(x, axis=1, keepdims=True)
    my = tf.reduce_mean(y, axis=1, keepdims=True)
    xm, ym = x - mx, y - my
    t1_norm = tf.nn.l2_normalize(xm, axis = 1)
    t2_norm = tf.nn.l2_normalize(ym, axis = 1)
    cosine = tf.losses.cosine_distance(t1_norm, t2_norm, axis = 1)
    return cosine

In [11]:
TIME_PERIODS = 1000
num_sensors = 1

model1 = Sequential()
model1.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(1000, 1)))
model1.add(Conv1D(64, 32, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model1.add(MaxPooling1D(3))
# model_m.add(Conv1D(32, 10, activation='relu'))
# model_m.add(MaxPooling1D(3))
model1.add(Conv1D(32, 16, activation='relu'))
model1.add(MaxPooling1D(3))
model1.add(Conv1D(16, 8, activation='relu'))
model1.add(MaxPooling1D(3))
model1.add(Flatten())
model1.add(Dense(1, activation='relu'))
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1000, 1)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 969, 64)           2112      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 323, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 308, 32)           32800     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 102, 32)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 95, 16)            4112      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 31, 16)           

In [12]:
model1.compile(loss=root_mean_squared_error,
                optimizer='adam', metrics=['mae'])

BATCH_SIZE = 32
EPOCHS = 100

history = model1.fit(X_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      # callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

2022-04-18 12:26:02.061759: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-18 12:26:02.081150: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199915000 Hz


Epoch 1/100


2022-04-18 12:26:04.044286: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-04-18 12:26:04.790013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


10/10 [==============================] - 7s 130ms/step - loss: 59.3077 - mae: 58.8785 - val_loss: 58.9212 - val_mae: 58.6515
Epoch 2/100
10/10 [==============================] - 0s 14ms/step - loss: 56.3249 - mae: 55.8711 - val_loss: 41.9383 - val_mae: 41.3449
Epoch 3/100
10/10 [==============================] - 0s 20ms/step - loss: 31.8007 - mae: 30.2529 - val_loss: 25.2300 - val_mae: 21.1526
Epoch 4/100
10/10 [==============================] - 0s 20ms/step - loss: 14.1253 - mae: 11.3922 - val_loss: 13.4395 - val_mae: 11.0768
Epoch 5/100
10/10 [==============================] - 0s 18ms/step - loss: 13.4133 - mae: 10.5873 - val_loss: 13.5187 - val_mae: 11.1953
Epoch 6/100
10/10 [==============================] - 0s 16ms/step - loss: 12.0951 - mae: 9.6211 - val_loss: 13.5369 - val_mae: 11.0991
Epoch 7/100
10/10 [==============================] - 0s 14ms/step - loss: 12.2576 - mae: 9.5595 - val_loss: 13.7893 - val_mae: 11.1770
Epoch 8/100
10/10 [==============================] - 0s 17ms/

In [13]:
pred = model1.predict(X_test)

In [14]:
root_mean_squared_error(np.array(y_test), np.array(pred))

<tf.Tensor: shape=(), dtype=float64, numpy=7.056020655694652>

In [15]:
s = len(pred)
pearsonr(np.array(pred).reshape(s), np.array(y_test).reshape(s))

(-0.12433648130620692, 0.22253484886997704)